Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# MONAI Registration Regularizers Overview

This notebook discusses details regarding `BendingEnergyLoss` and `DiffusionLoss`.

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly"

## Setup imports

In [ ]:
import torch
from monai.config import print_config
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.transforms import AsDiscrete, Compose

print_config()


def print_tensor(name, t):
    print(f"{name}: {t.numpy().tolist()} shape: {tuple(t.shape)}")

## Definitions

In this section, we provide the definitions of diffusion loss and bending energy loss in the 3D case. 
$\def\p{{\mathbf p}}$
$\def\u{{\mathbf u}}$

Let $\Omega \subseteq \mathbb{R}^3$ be the image domain on which moving and fixed images are defined.

The diffusion loss is given by $$\begin{align*} R_\mathrm{diffusion} & = \sum_{\p \in \Omega} ||\nabla \u (\p)||^2 \\ & = \sum_{\p \in \Omega} \left [ \left( \frac{\partial \u (\p)}{\partial x} \right)^2 + \left( \frac{\partial \u (\p)}{\partial y} \right)^2 + \left( \frac{\partial \u (\p)}{\partial z} \right)^2 \right ], \end{align*}$$

where $\nabla \u (\p)$ denote the spatial gradient of the dense displacement field (DDF) $\u$ at voxel $\p$.

The bending energy loss is given by $$\begin{align*} R_\mathrm{bending} & = \sum_{\p \in \Omega} ||\nabla^2 \u (\p)||^2 \\ & = \sum_{\p \in \Omega} \left [ \left( \frac{\partial^2 \u (\p)}{\partial x^2} \right)^2 + \left( \frac{\partial^2 \u (\p)}{\partial y^2} \right)^2 + \left( \frac{\partial^2 \u (\p)}{\partial z^2} \right)^2 + \left( \frac{\partial^2 \u (\p)}{\partial x \partial y} \right)^2 + \left( \frac{\partial^2 \u (\p)}{\partial y \partial z} \right)^2 + \left( \frac{\partial^2 \u (\p)}{\partial z \partial x} \right)^2 \right ], \end{align*}$$

where $\nabla^2 \u (\p)$ denote the Hessian matrix of the dense displacement field (DDF) $\u$ at voxel $\p$.

Central finite difference is used to estimate the spatial gradients, i.e.,

$$\frac{\partial \u (\p)}{\partial x} \approx \frac{1}{2} [\u ((p_x + 1, p_y, p_z)) - \u ((p_x - 1, p_y, p_z)) ],$$

$$\frac{\partial \u (\p)}{\partial y} \approx \frac{1}{2} [\u ((p_x, p_y + 1, p_z)) - \u ((p_x, p_y - 1, p_z)) ],$$

$$\frac{\partial \u (\p)}{\partial z} \approx \frac{1}{2} [\u ((p_x, p_y, p_z + 1)) - \u ((p_x, p_y, p_z - 1)) ].$$

The second order spatial gradients are estimated similarly.